In [ ]:
from pathlib import Path
from lib import run_experiment
import numpy as np

In [ ]:
experiments_path = Path('experiments')

# Experiment 1
---
Dimension: 20

Number of clusters: 2

T parameter (distance between centroids): [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

P Minkowski parameter: [0.2, 0.5, 1, 2]

M number of repeats: 1000

N number of points: [100, 500, 1000]

In [ ]:
dimension = 20
n_clusters = 2
T_parameter = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
minkowski_parameter = [0.2, 0.5, 1, 2]
repeats = 1000
n_points = [100, 500, 1000]

In [ ]:
sigma_list = [1, 1]
prob = 0.5
mu_list = [np.array([x + [0] * (dimension-2)])
            for x in [[-2, 0], [2, 0]]]

In [ ]:
T_parameter = [0.1]
minkowski_parameter = [0.2, 0.5, 1, 2]
repeats = 1
n_points = [100]

In [ ]:
for points in n_points:
    experiment_name = f'Experiment 1, N_points:{points}'
    output_path = experiments_path / f'experiment_1_{points}'

    run_experiment(
        dimension=dimension,
        n_clusters=n_clusters,
        distance_parameters=T_parameter,
        minkowski_parameters=minkowski_parameter,
        repeats=repeats,
        n_points=points,
        sigma_list=sigma_list,
        prob=prob,
        mu_list=mu_list,
        experiment_name=experiment_name,
        output_path=output_path
    )

# Experiment 2
---
Dimension: 20

Number of clusters: 3

T parameter (distance between centroids): [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

P Minkowski parameter: [0.2, 0.5, 1, 2]

M number of repeats: 1000

N number of points: [100, 500, 1000]

In [ ]:
dimension = 20
n_clusters = 3
T_parameter = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
minkowski_parameter = [0.2, 0.5, 1, 2]
repeats = 1000
n_points = [100, 500, 1000]

In [ ]:
sigma_list = [1, 1, 1]
prob = 1/3
mu_list = [np.array([x + [0] * (dimension-3)])
            for x in [[3, 0, 0], [0, 3, 0], [0, 0, 3]]]

In [ ]:
T_parameter = [0.1]
minkowski_parameter = [0.2, 0.5, 1, 2]
repeats = 1
n_points = [100]

In [ ]:
for points in n_points:
    experiment_name = f'Experiment 1, N_points:{points}'
    output_path = experiments_path / f'experiment_1_{points}'

    run_experiment(
        dimension=dimension,
        n_clusters=n_clusters,
        distance_parameters=T_parameter,
        minkowski_parameters=minkowski_parameter,
        repeats=repeats,
        n_points=points,
        sigma_list=sigma_list,
        prob=prob,
        mu_list=mu_list,
        experiment_name=experiment_name,
        output_path=output_path
    )